In [4]:
import pytorch_kinematics as pk
import torch
urdf = f"/home/pine/RoboTwin/aloha_maniskill_sim/urdf/arx5_description_isaac.urdf"
# urdf = f"/data1/hydeng/Keguide_RoboTwin/aloha_maniskill_sim/urdf/arx5_description_isaac.urdf"
# there are multiple natural end effector links so it's not a serial chain
chain = pk.build_chain_from_urdf(open(urdf, mode="rb").read())
pk_chain_left = pk.build_serial_chain_from_urdf(
                open(urdf, mode="rb").read(), 
                end_link_name="fl_base_link",
                root_link_name="footprint"
            )
print(pk_chain_left)
# visualize the frames (the string is also returned)
kinematic_tree = chain.print_tree()



footprint
└── fl_base_link

footprint
├── base_link
│   ├── inertial_link
│   ├── right_wheel_link
│   ├── left_wheel_link
│   ├── fl_castor_link
│   │   └── fl_wheel_link
│   ├── fr_castor_link
│   │   └── fr_wheel_link
│   ├── rr_castor_link
│   │   └── rr_wheel_link
│   └── rl_castor_link
│       └── rl_wheel_link
├── box1_Link
│   ├── box2_Link
│   └── camera_base_link
│       └── camera_link1
│           └── camera_link2
├── fl_base_link
│   └── fl_link1
│       └── fl_link2
│           └── fl_link3
│               └── fl_link4
│                   └── fl_link5
│                       └── fl_link6
│                           ├── left_camera
│                           ├── fl_link7
│                           └── fl_link8
├── fr_base_link
│   └── fr_link1
│       └── fr_link2
│           └── fr_link3
│               └── fr_link4
│                   └── fr_link5
│                       └── fr_link6
│                           ├── right_camera
│                           ├── fr_link7


In [5]:
# extract a specific serial chain such as for inverse kinematics
serial_chain = pk.SerialChain(chain, "fl_link6", "fl_base_link")
serial_chain.print_tree()

fl_base_link
└── fl_link1
    └── fl_link2
        └── fl_link3
            └── fl_link4
                └── fl_link5
                    └── fl_link6



'fl_base_link\n└── fl_link1\n    └── fl_link2\n        └── fl_link3\n            └── fl_link4\n                └── fl_link5\n                    └── fl_link6\n'

In [9]:
def get_transform_base_to_fl(urdf_data):
    """
    使用 PyTorch-Kinematics 获取 base_link -> fl_base_link 的 4x4 变换矩阵。
    注意: base_link 和 fl_base_link 父节点都是 footprint，需要先分别求它们相对于 footprint 的变换。
    """
    # 1) 构造 'footprint' -> 'base_link' 链
    chain_base = pk.build_serial_chain_from_urdf(
        urdf_data,
        end_link_name="base_link",
        root_link_name="footprint"
    )
    # 由于是固定关节(或者没有可动关节)，这里传一个空列表即可
    # end_only=False, 获取整个链所有 link 的 Transform; 也可 end_only=True 后直接取返回的 Transform3d
    ret_base = chain_base.forward_kinematics([], end_only=False)
    # ret_base 是一个 dict, ret_base["base_link"] 就是 footprint->base_link 的 Transform3d
    tf_base = ret_base["base_link"].get_matrix()  # (1, 4, 4) 形状的张量
    
    # 2) 构造 'footprint' -> 'fl_base_link' 链
    chain_fl = pk.build_serial_chain_from_urdf(
        urdf_data,
        end_link_name="fr_base_link",
        root_link_name="footprint"
    )
    ret_fl = chain_fl.forward_kinematics([], end_only=False)
    tf_fl = ret_fl["fr_base_link"].get_matrix()  # (1, 4, 4)
    
    # 3) 做变换运算: base_link -> fl_base_link = inv(footprint->base_link) @ (footprint->fl_base_link)
    # tf_base 和 tf_fl 的 shape 都是 (1,4,4)，先 squeeze 一下拿到 (4,4)，或直接保持(1,4,4)也行
    tf_base_44 = tf_base.squeeze(0)  # shape = (4,4)
    tf_fl_44 = tf_fl.squeeze(0)      # shape = (4,4)
    T_base_to_fl = torch.inverse(tf_base_44) @ tf_fl_44  # (4,4) 的张量
    
    return T_base_to_fl, tf_base_44, tf_fl_44

In [11]:
T_base_to_fl, tf_base_44, tf_fl_44 = get_transform_base_to_fl(open(urdf, mode="rb").read())
print("Transform [base_link -> fl_base_link] =\n", tf_base_44)
print("Transform [base_link -> fl_base_link] =\n", tf_fl_44)
print("Transform [base_link -> fr_base_link] =\n", T_base_to_fl)

Transform [base_link -> fl_base_link] =
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.1500],
        [0.0000, 0.0000, 0.0000, 1.0000]])
Transform [base_link -> fl_base_link] =
 tensor([[ 1.0000,  0.0000,  0.0000,  0.2330],
        [ 0.0000,  1.0000,  0.0000, -0.3000],
        [ 0.0000,  0.0000,  1.0000,  0.7775],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
Transform [base_link -> fr_base_link] =
 tensor([[ 1.0000,  0.0000,  0.0000,  0.2330],
        [ 0.0000,  1.0000,  0.0000, -0.3000],
        [ 0.0000,  0.0000,  1.0000,  0.6275],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
